# 08. Geographic Heatmaps (National & State-wise)

This notebook generates heatmaps to visualize the intensity of Aadhaar activities across India. 
We use two approaches:
1. **Statistical Heatmaps**: Matrix showing States vs. various Metrics.
2. **Geographic Choropleth Maps**: Visualizing data on a Map of India.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import requests

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

os.makedirs('../../visualizations', exist_ok=True)
print("Environment ready.")

## 1. Load and Clean State Data
We need to consolidate state names (e.g., handling '&' vs 'and') for clean mapping.

In [ ]:
df = pd.read_csv('../../processed_data/geographic_data.csv')

def clean_state(state):
    if pd.isna(state): return state
    state = state.replace(' & ', ' and ')
    if state == 'Orissa': return 'Odisha'
    return state

df['state'] = df['state'].apply(clean_state)

# Aggregating to State level for heatmaps
state_metrics = df.groupby('state').agg({
    'total_enrollments': 'sum',
    'total_updates': 'sum',
    'age_0_5': 'sum',
    'age_18_greater': 'sum'
}).reset_index()

print(f"Aggregated data for {len(state_metrics)} states.")

## 2. Statistical Heatmap: State vs Metrics
This shows which states are most active across different enrollment and update categories.

In [ ]:
# Normalize data for better heatmap visualization
heatmap_df = state_metrics.set_index('state')
heatmap_df_norm = (heatmap_df - heatmap_df.min()) / (heatmap_df.max() - heatmap_df.min())

sns.heatmap(heatmap_df_norm, annot=False, cmap='YlGnBu', linewidths=0.5)
plt.title('Relative Intensity of Aadhaar Metrics across States')
plt.savefig('../../visualizations/08_state_metrics_intensity_heatmap.png')
plt.show()

## 3. Geographic Heatmap (Choropleth)
Visualizing Total Updates across the Map of India.

In [ ]:
print("Fetching India GeoJSON...")
india_geojson_url = "https://raw.githubusercontent.com/geohacker/india/master/state/india_state.geojson"
response = requests.get(india_geojson_url)
india_geojson = response.json()

fig = px.choropleth(
    state_metrics,
    geojson=india_geojson,
    featureidkey='properties.NAME_1',
    locations='state',
    color='total_updates',
    color_continuous_scale="Viridis",
    title='Heatmap: Aadhaar Update Intensity across India',
    labels={'total_updates': 'Total Updates'}
)

fig.update_geos(fitbounds="locations", visible=False)
fig.write_image('../../visualizations/08_national_update_heatmap.png')
fig.show()

## 4. State-wise Heatmap (District Level)
Visualizing intensity within a selected state (Example: Uttar Pradesh).

In [ ]:
selected_state = 'Uttar Pradesh' # You can change this to any state
state_df = df[df['state'] == selected_state].groupby('district').sum(numeric_only=True).reset_index()

plt.figure(figsize=(12, 10))
sns.barplot(data=state_df.sort_values('total_updates', ascending=False).head(20), 
            x='total_updates', y='district', palette='rocket')
plt.title(f'Update Intensity by District in {selected_state} (Top 20)')
plt.savefig(f'../../visualizations/08_{selected_state.lower().replace(" ", "_")}_district_heatmap.png')
plt.show()